In [30]:
'''USING QUANDL TO GATHER FINANCIAL DATA
    - https://www.quandl.com/tools/api'''
import quandl

'''LIBRARY NUMPY -- USED FOR MANIPULATING ARRAYS AND DATAFRAMES'''
import numpy as np


'''USING SKLEARN TO IMPORT MODELS AND TO TEST OUR DATA
    - LINEAR MODEL --> LINEAR REGRESSION
    - SUPPORT VECTOR MACHINE --> SUPPORT VECTOR REGRESSION
    TRAIN TEST SPLIT --> TO TRAIN AND TEST DATA'''
from sklearn.linear_model import LinearRegression 
from sklearn.svm import SVR 
from sklearn.model_selection import train_test_split



print("[SUCCESS]")

[SUCCESS]


In [3]:
# We begin by getting our data stock -- AMAZON
amazon_df = quandl.get('WIKI/AMZN')
print(amazon_df.head())

             Open   High    Low  Close     Volume  Ex-Dividend  Split Ratio  \
Date                                                                          
1997-05-16  22.38  23.75  20.50  20.75  1225000.0          0.0          1.0   
1997-05-19  20.50  21.25  19.50  20.50   508900.0          0.0          1.0   
1997-05-20  20.75  21.00  19.63  19.63   455600.0          0.0          1.0   
1997-05-21  19.25  19.75  16.50  17.13  1571100.0          0.0          1.0   
1997-05-22  17.25  17.38  15.75  16.75   981400.0          0.0          1.0   

            Adj. Open  Adj. High  Adj. Low  Adj. Close  Adj. Volume  
Date                                                                 
1997-05-16   1.865000   1.979167  1.708333    1.729167   14700000.0  
1997-05-19   1.708333   1.770833  1.625000    1.708333    6106800.0  
1997-05-20   1.729167   1.750000  1.635833    1.635833    5467200.0  
1997-05-21   1.604167   1.645833  1.375000    1.427500   18853200.0  
1997-05-22   1.437500   1.

In [4]:
'''ONLY NEED CLOSING PRICE -- REWRITE OUR DATAFRAME TO FIT THOSE PARAMATERS'''

amazon_df = amazon_df[['Adj. Close']]
print(amazon_df.head())

            Adj. Close
Date                  
1997-05-16    1.729167
1997-05-19    1.708333
1997-05-20    1.635833
1997-05-21    1.427500
1997-05-22    1.395833


In [9]:
'''CREATE A VARIABLE TO STORE THE NUMBER OF DAYS OUT TO PREDICT
    - FOR THIS WE USE 15 DAYS
    - CAN BE CHANGED TO ANY AMOUNT'''
nday = 15 

'''SINCE WE ARE DOING A PREDICTIVE ANALYSIS, CREATE A NEW COLUMN TO STORE THE TARGET PREDICTION'''
amazon_df['Stock Prediction'] = amazon_df[['Adj. Close']].shift(-nday)

'''SHOW OUR DATAFRAME'''
print(f"Showing the amazon_df.head()\n \n {amazon_df.head()}")
print()
print(f"Showing the amazon_df.tail()\n \n {amazon_df.tail()}")


Showing the amazon_df.head()
 
             Adj. Close  Stock Prediction
Date                                    
1997-05-16    1.729167          1.687500
1997-05-19    1.708333          1.583333
1997-05-20    1.635833          1.541667
1997-05-21    1.427500          1.604167
1997-05-22    1.395833          1.583333

Showing the amazon_df.tail()
 
             Adj. Close  Stock Prediction
Date                                    
2018-03-21     1581.86               NaN
2018-03-22     1544.10               NaN
2018-03-23     1495.56               NaN
2018-03-26     1555.86               NaN
2018-03-27     1497.05               NaN


In [10]:
'''CREATE A NEW *INDEPENDENT* DATASET TO TRAIN LINEAR REGRESSION MODEL
    1. CONVERT DATAFRAME TO NUMPY ARRAY USING ONLY Adj. Close COLUMN
    2. REMOVE LAST N NUMBER OF ROWS'''
#1.
x_data = np.array(amazon_df.drop(['Stock Prediction'], 1))
#2.
x_data = x_data[:-nday]


print(f"Numpy Array \n {x_data}")

Numpy Array 
 [[   1.72916667]
 [   1.70833333]
 [   1.63583333]
 ...
 [1500.25      ]
 [1523.61      ]
 [1537.64      ]]


In [15]:
'''CREATE THE NEW *DEPENDENT* DATASET -- AKA THE TARGET DATA
    1. CONVERT DATAFRAME TO NUMPY ARRAY FROM Stock Prediction COLUMN
    2. REMOVE THE LAST N NUMBER OF ROWS
    3. MAKE SURE THERE ARE NO NaN VALUES'''
#1
y_data = np.array(amazon_df['Stock Prediction'])

#2
y_data = y_data[:-nday]

#3
print(f"Target Data \n {y_data}")


Target Data 
 [1.68750000e+00 1.58333333e+00 1.54166667e+00 ... 1.49556000e+03
 1.55586000e+03 1.49705000e+03]


In [16]:
'''SET UP OUR TRAIN TEST SPLIT
    - x_data, y_data --> datasets we are using
    - test_size is split to 80% training and 20% testing'''

'''TESTING DATA IS SET -- MOVE ON TO THE MODELS'''
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

In [18]:
'''CREATE SUPPORT VECTOR MACHINE (REGRESSION) MODEL
    - MODEL ATTEMPTS TO PREDICT A STOCK PRICE IN THE FUTURE
    
    1. USING 3 PARAMETERS
        - KERNEL (TYPE OF MATHEMATICAL FUNCTION TO USE)
            - USING RBF FOR THIS MODEL -- RADIAL BASIS FUNCTION -- GAUSSIAN FUNCTION
            - https://scikit-learn.org/stable/modules/svm.html
            - https://www.cs.princeton.edu/sites/default/files/uploads/saahil_madge.pdf
            
        - C
            - HELPS WITH THE MISCLASSIFICATION OF TRAINING EXAMPLES
            - CHOOSING A LOW INT CREATES A SMOOTH DECISION
            - CHOOSING A HIGH INT CREATES THE CLASSIFICATION OF DATA MORE ACCURATELY
            
        - GAMMA
            - DECIDING FACTOR OF HOW MUCH INFLUENCE A SINGLE TRAINING EXAMPLE HAS
            
        - USED THE FOLLOWING TO CORRECTLY CHOOSE A PROPER C AND GAMMA VALUES
            - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV
        
    2. THEN REFIT THE DATA WITH THE SVM MACHINE USING .fit()
        - .fit(1, 2)
            - 1 --> TRAINING VECTOR WHERE N SAMPLES IS THE NUMBER OF SAMPLES
            - 2 --> TARGET IS RELATIVE TO X FOR REGRESSION
            
    '''

#1
svr_machine = SVR(kernel = 'rbf', C = 1e3, gamma = 0.1)

#2
svr_machine.fit(x_train, y_train)

SVR(C=1000.0, gamma=0.1)

In [22]:
'''GET THE SCORE OF THE SVM MODEL -- REGRESSION
    1. SCORE Returns the coefficient of determination R^2 of the prediction.
        - BEST POSSIBLE SCORE IS 1.0 -- CAN BE NEGATIVE
        - .score(1, 2) PARAMETERS
            - 1 --> TRAINING VECTOR WHERE N SAMPLES IS THE NUMBER OF SAMPLES
            - 2 --> TARGET IS RELATIVE TO X FOR REGRESSION'''


#1
SVM_score = svr_machine.score(x_test, y_test)


print(f"SVM (Regression)  Confidence: {SVM_score}")

SVM (Regression)  Confidence: 0.9698616099122792


In [23]:
'''1. CREATE LINEAR REGRESSION MODEL
   2. TRAIN MODEL'''

#1
lnr = LinearRegression()

#2
lnr.fit(x_train, y_train)

LinearRegression()

In [27]:
'''GET THE SCORE OF THE LINEAR REGRESSION MODEL
        1. SCORE Returns the coefficient of determination R^2 of the prediction.
        - BEST POSSIBLE SCORE IS 1.0 -- CAN BE NEGATIVE
        - .score(1, 2) PARAMETERS
            - 1 --> TRAINING VECTOR WHERE N SAMPLES IS THE NUMBER OF SAMPLES
            - 2 --> TARGET IS RELATIVE TO X FOR REGRESSION'''

#1
lnr_confidence = lnr.score(x_test, y_test)



print(f"Linear Confidence: {lnr_confidence}")
print('Since model score is so close to 1, it is better to predict the Amazon Stock price.')

Linear Confidence: 0.9950907863827594
Since model score is so close to 1, it is better to predict the Amazon Stock price.


In [28]:
'''MODELS ARE TRAINED AND TESTED -- TIME FOR PREDICTIONS

    1. CREATE A NEW NUMPY ARRAY BY DROPPING THE STOCK PREDICTION AND ADDING IN OUR N DAYS OUT FROM THE BEGINNING
    2. PRED
        - DATA TO BE USED TO MAKE PREDICTIONS

    '''
#1
pred = np.array(amazon_df.drop(['Stock Prediction'], 1))[-nday:]




print(pred)

[[1545.  ]
 [1551.86]
 [1578.89]
 [1598.39]
 [1588.18]
 [1591.  ]
 [1582.32]
 [1571.68]
 [1544.93]
 [1586.51]
 [1581.86]
 [1544.1 ]
 [1495.56]
 [1555.86]
 [1497.05]]


In [32]:
'''PRINT PREDICTIONS FOR BOTH MODELS USING .predict()
    - THIS WILL PERFORM CLASSIFICATION ON SAMPLES IN OUR X_DATA'''

'''LINEAR REGRESSION PREDICTION'''
lnr_prediction = lnr.predict(pred)
print('Linear Regression Model Prediction Prices -- 15 days out')
for i in lnr_prediction:
    print(f"Stock Price: ${i}")
print()

'''SUPPORT VECTOR MACHINE PREDICTION'''
SVM_prediction = svr_machine.predict(pred)
print('Support Vector Machine (Regression) Prediction Prices -- 15 days out')
for i in SVM_prediction:
    print(f"Stock Price: ${i}")

Linear Regression Model Prediction Prices -- 15 days out
Stock Price: $1591.9721209682748
Stock Price: $1599.0484922853348
Stock Price: $1626.9310457401662
Stock Price: $1647.0460954023638
Stock Price: $1636.5140617074385
Stock Price: $1639.4230073508947
Stock Price: $1630.4692313986961
Stock Price: $1619.4936350701946
Stock Price: $1591.8999130976927
Stock Price: $1634.791388223548
Stock Price: $1629.9947225348697
Stock Price: $1591.0437340607887
Stock Price: $1540.9727335170414
Stock Price: $1603.1746563186061
Stock Price: $1542.509729619435

Support Vector Machine (Regression) Prediction Prices -- 15 days out
Stock Price: $695.9695210831736
Stock Price: $692.3968047447939
Stock Price: $692.3968034150889
Stock Price: $692.3968034150889
Stock Price: $692.3968034150889
Stock Price: $692.3968034150889
Stock Price: $692.3968034150889
Stock Price: $692.3968034150889
Stock Price: $696.3553485110584
Stock Price: $692.3968034150889
Stock Price: $692.3968034150889
Stock Price: $704.7897384842

**RESOURCCES THAT HELPED ME**

1. https://www.analyticsvidhya.com/blog/2017/09/understaing-support-vector-machine-example-code/
2. https://towardsdatascience.com/introduction-to-linear-regression-in-python-c12a072bedf0
3. https://www.quora.com/How-do-I-select-SVM-kernels
4. https://www.cs.princeton.edu/sites/default/files/uploads/saahil_madge.pdf
5. https://journalofbigdata.springeropen.com/articles/10.1186/s40537-020-00333-6
6. https://www.researchgate.net/post/Diffference_between_SVM_Linear_polynmial_and_RBF_kernel
7. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV
8. https://scikit-learn.org/stable/modules/svm.html

